# Задание

### Главные задачи, которые должен выполнять скрипт:

a.	Извлекать реплики с приветствием – где менеджер поздоровался.

b.	Извлекать реплики, где менеджер представил себя.

c.	Извлекать имя менеджера.

d.	Извлекать название компании.

e.	Извлекать реплики, где менеджер попрощался.

f.	Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом


### Рекомендации:

a.	Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!

b.	Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.

c.	Для выполнения задачи можно использовать любые библиотеки и NLP модели.

d.	Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей



In [1]:
import pandas as pd
import numpy as np
import spacy
import re

In [2]:
from razdel import tokenize
import pymorphy2
from nltk.corpus import stopwords
import nltk
from gensim.corpora.dictionary import Dictionary

In [3]:
#!python -m spacy download ru_core_news_sm
#nltk.download('stopwords')
#!pip install nltk.tag

In [4]:
stopword_ru = stopwords.words('russian')
#len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [5]:
data = pd.read_csv('test_data - Копия test_data.csv')
data

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [6]:
for column in data.columns[:-1]:
    print(f'Уникальных значений в колонке {column} = {len(data[column].unique())}')

Уникальных значений в колонке dlg_id = 6
Уникальных значений в колонке line_n = 143
Уникальных значений в колонке role = 2


In [7]:
nlp = spacy.load('ru_core_news_sm')

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text) #Можно добавть [0-9] Если нужно убрать цифры.
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

def lemmatization_spacy(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

        на выходе лист отлемматизированых токенов
    '''  
   
    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    #[1]
    words=[w.text for w in nlp(text)]
    
    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [9]:
%%time
#Запускаем очистку текста.
data['text_cline'] = data['text'].apply(lambda x: clean_text(x), 1)

Wall time: 11 ms


C:\Users\maxim\AppData\Local\Temp/ipykernel_6380/2776340022.py:15: FutureWarning: Possible nested set at position 33
  text = re.sub("[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text) #Можно добавть [0-9] Если нужно убрать цифры.


In [10]:
%%time
#Запускаем лемматизацию текста.
data['text_lim'] = data['text_cline'].apply(lambda x: lemmatization(x), 1)

Wall time: 170 ms


In [11]:
%%time
#Запускаем лемматизацию текста 2.
data['text_lim_spacy'] = data['text_cline'].apply(lambda x: lemmatization_spacy(x), 1)

Wall time: 2.74 s


In [12]:
data.head()

,dlg_id,line_n,role,text,text_cline,text_lim,text_lim_spacy
0,0,0,client,Алло,алло,[алло],[алло]
1,0,1,manager,Алло здравствуйте,алло здравствуйте,"[алло, здравствуйте]","[алло, здравствуйте]"
2,0,2,client,Добрый день,добрый день,"[добрый, день]","[добрый, день]"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,меня зовут ангелина компания диджитал бизнес з...,"[звать, ангелина, компания, диджитал, бизнес, ...","[звать, ангелина, компания, диджитал, бизнес, ..."
4,0,4,client,Ага,ага,[ага],[ага]


#### Функция lemmatization_spacy работает медленне lemmatization, но получает те же рез-ты. Будем успользовать больее быструю функуцию для лимитизации.

# a. Извлекаем реплики с приветствием – где менеджер поздоровался.

In [13]:
def greeting(text):
    '''
    Поиск в тексте по ключевым словам приветвия
    '''
    greetings=['здравствуйте', 'зравствуй', 'добрый', 
               'приветвую', 'салют' 'привет', 'hi', 'хай']
    
    for word in text:
        if word in greetings:
            return True
    

In [14]:
# Запускаем поиск приветвия.  
#data['greetings'] = data['text_lim'].apply(lambda x: greeting(x), 1)

In [15]:
def greetings_list(data, manager= True, first=10):
    '''
    Функция выводит список индексов строк, в которых здоровались
    
    параметры функции:
        df - DataFrame 
        manager - Сказал менеджер. boolean
        first - ищим в первых N строк диалога.
    '''
    
    df=data.copy()
    df['greetings'] = df['text_lim'].apply(lambda x: greeting(x), 1)
    
    if manager:
        return df.loc[(df['greetings']== True) & (df['role']== 'manager') & (df['line_n']<=first)].index.tolist()
    else:
        return df.loc[(df['greetings']== True) & (df['line_n']<=first)].index.tolist()
    

In [16]:
list_greetins = greetings_list(data)

data['greetings']=None
data.loc[list_greetins, 'greetings'] = True

data.loc[data['greetings']==True, ['dlg_id','line_n','role', 'text', 'greetings']]

,dlg_id,line_n,role,text,greetings
1,0,1,manager,Алло здравствуйте,True
110,1,1,manager,Алло здравствуйте,True
166,2,2,manager,Алло здравствуйте,True
250,3,1,manager,Алло дмитрий добрый день,True
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,True


# b. Извлекаем реплики, где менеджер представил себя.
# c. Извлекаем имя менеджера

In [17]:
from nltk import pos_tag, word_tokenize
from nltk.tag import UnigramTagger
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger_ru')

In [18]:
def manager_name (text, prob_thresh=0.4, only_name=False):
    '''
    [0] Поиск имени в тексте
    [1] определить пред-ся ли человек
    
    '''
    morph = pymorphy2.MorphAnalyzer()
    
    words_for_meneger= set(['я', 'звать', 'это']) # Слова, котрые должны стоять рядом с именем. Для проверки на представления.
    
    if isinstance(text, list):
        text_= " ".join(text)
        
    # [0] 
    for word in nltk.word_tokenize(text_):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:   
                
                # [1]
                if only_name==False: 
                    overlap = words_for_meneger & set(text)
                    if overlap:
                        word_for_meneger=overlap.pop()
                        if abs(text.index(word)-text.index(word_for_meneger))==1: #Имя и ключевое слово должны быть рядом!
                            return word
                else:
                    #print(f'Имя {word}, с вероятностью {p.score}')
                    return word

In [19]:
%%time
# Запускаем поиск имен. И записываем имена пред-я в новую колонку. Будет долго...
data['manager_name'] = data['text_lim'].apply(lambda x: manager_name(x,only_name=False), 1)

Wall time: 1min 40s


In [20]:
# Имена менеджеров.
data.loc[(data['manager_name'].notna()) & (data['role']== 'manager'), ['dlg_id','line_n','role', 'text', 'manager_name']]

,dlg_id,line_n,role,text,manager_name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,максим
338,5,1,manager,Да это анастасия,анастасия


# d. Извлекать название компании.

In [21]:
#!pip install natasha

In [22]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [23]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [28]:
def return_orgs(data):
    doc=Doc(data)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    jam=list()
    for item2 in doc.spans:
        if item2.type == "ORG":
            if item2.text not in ['Алло', 'Ага']:
                print(item2)
                jam.append(item2.text)
                return str(jam)

In [29]:
data['organisation']=data['text'].apply(return_orgs)

DocSpan(stop=15, type='ORG', text='Диджитал бизнес', tokens=[...])
DocSpan(stop=21, type='ORG', text='Транспортная компания', tokens=[...])


In [30]:
data.loc[data['organisation'].notna(), ['dlg_id','line_n','role', 'text', 'organisation']]

,dlg_id,line_n,role,text,organisation
177,2,13,manager,Диджитал бизнес,['Диджитал бизнес']
272,3,23,client,Транспортная компания,['Транспортная компания']


##### К сожалению данный модуль не нашел названия компаний в середине текста. Поэтому попробуем "вручную":

In [31]:
def org_name(text):
    """
    Делим на токены и ищем сочетания токенов
    """
    Tokens = nltk.word_tokenize(text)
    output = list(nltk.bigrams(Tokens))
    if ('диджитал', 'бизнес') in output:
        return True
    if ('транспортная', 'компания') in output:
        return True

In [32]:
data['organisation']=data['text_cline'].apply(org_name)
data.loc[data['organisation'].notna(), ['dlg_id','line_n','role', 'text', 'organisation']]

,dlg_id,line_n,role,text,organisation
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,True
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,True
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,True
177,2,13,manager,Диджитал бизнес,True
272,3,23,client,Транспортная компания,True
273,3,24,manager,Транспортная компания хорошо а у вас уже вот в...,True


# e.	Извлекать реплики, где менеджер попрощался.

In [33]:
def key_words_goodbye(text):
    '''
    Поиск в тексте по ключевым словам прощания
    '''
    goodbyes=['свидание', 'хорошо', 'прощай', 'добрый']
    
    for word in text:
        if word in goodbyes:
            return True

In [34]:
def say_goodbye(data, manager=True, last_str=10):
    '''
    Функция выводит список индексов строк, в которыз прощались
    
    параметры функции:
        df - DataFrame 
        manager - Сказал менеджер. boolean
        last_str - ищим в в последних N строк диалога.
    '''
    df=data.copy() 
    df['key_words_goodbye']=df['text_lim'].apply(key_words_goodbye)
    
    data_goodbay= pd.DataFrame()
    for i in df['dlg_id'].unique().tolist():
        first= len(df.loc[df['dlg_id']==i])-last_str  
        last = len(df.loc[df['dlg_id']==i])
        if manager:
            data_ = df.loc[df['dlg_id']==i].loc[(df['line_n'].between(first, last))& (df['role']== 'manager')]
        else:
            data_ = df.loc[df['dlg_id']==i].loc[df['line_n'].between(first, last)]
        
        data_goodbay = pd.concat([data_goodbay, data_])
    return data_goodbay.loc[df['key_words_goodbye'].notna()].index.tolist()

In [35]:
list_goodbay=say_goodbye(data)

data['goodbye']=None
data.loc[list_goodbay, 'goodbye'] = True

data.loc[data['goodbye']==True, ['dlg_id','line_n','role', 'text', 'goodbye']]

,dlg_id,line_n,role,text,goodbye
108,0,108,manager,Всего хорошего до свидания,True
163,1,54,manager,До свидания,True
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,True
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,True
479,5,142,manager,Ну до свидания хорошего вечера,True


# f. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом

In [75]:
def good_manager(data, only_good=True):
    '''
    Проверка требований к менеджеру. Если only_good , то будет печать только "хороших" менеджеров, в противном случае всех
    с указанием недостатков.
    '''

    for i in data['dlg_id'].unique().tolist():
    
        df_ = data.loc[data['dlg_id']==i]
    
        try:
            manager_name  = df_.loc[df_['manager_name'].notna()]['manager_name'].tolist()[0]
        except IndexError:
            manager_name = ''
            
        if only_good:
            if len(df_.loc[df_['greetings']==True]) > 0 and (len(df_.loc[df_['goodbye']==True]) > 0):
                print(f'В диалоге {i} менеджер {manager_name} поздаровался и попращался')
        else:
            if len(df_.loc[df_['greetings']==True]) > 0 and (len(df_.loc[df_['goodbye']==True]) > 0):
                print(f'В диалоге {i} менеджер {manager_name} поздаровался и попращался - МОЛОДЕЦ!')
            elif len(df_.loc[df_['greetings']==True]) > 0 and (len(df_.loc[df_['goodbye']==True]) == 0):
                print(f'В диалоге {i} менеджер {manager_name} поздаровался, но не попращался')
            elif len(df_.loc[df_['greetings']==True]) == 0 and (len(df_.loc[df_['goodbye']==True]) > 0):
                print(f'В диалоге {i} менеджер {manager_name} попращался, но не поздаровался')
    
    return print('-'*50,'END','-'*50)

In [77]:
good_manager(data, only_good=True)

В диалоге 0 менеджер ангелина поздаровался и попращался
В диалоге 1 менеджер ангелина поздаровался и попращался
В диалоге 3 менеджер максим поздаровался и попращался
-------------------------------------------------- END --------------------------------------------------


In [78]:
# На всякий случай список зависимостей

#!pip freeze > requirements.txt